# Gemini - Comprehensive Data Analysis Pipeline
## Advanced Analytics, Visualization & Insights

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✓ All libraries imported successfully!")

## 1️⃣ Data Loading & Initial Exploration

In [ ]:
# Load your dataset (adjust filename as needed)
# Replace 'your_data.csv' with your actual filename
df = pd.read_csv('your_data.csv')

print("="*70)
print("DATASET OVERVIEW")
print("="*70)
print(f"\nTotal Records: {len(df):,}")
print(f"Total Columns: {len(df.columns)}")
print(f"\nDataset Shape: {df.shape}")
print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n" + "="*70)
print("COLUMN INFORMATION")
print("="*70)
print(f"\nColumns ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col:30s} - {df[col].dtype}")

print("\n" + "="*70)
print("FIRST 10 ROWS")
print("="*70)
display(df.head(10))

## 2️⃣ Data Quality Assessment

In [ ]:
# Missing values analysis
print("="*70)
print("MISSING VALUES ANALYSIS")
print("="*70 + "\n")

missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
    'Data_Type': df.dtypes
})

missing_data = missing_data.sort_values('Missing_Count', ascending=False)
display(missing_data)

# Visualize missing data
if missing_data['Missing_Count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_cols = missing_data[missing_data['Missing_Count'] > 0]
    plt.barh(missing_cols['Column'], missing_cols['Missing_Percentage'], color='coral')
    plt.xlabel('Missing Percentage (%)', fontsize=12)
    plt.title('Missing Data by Column', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('01_missing_data_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 01_missing_data_analysis.png")
else:
    print("\n✓ No missing values detected!")

## 3️⃣ Statistical Summary & Distribution Analysis

In [ ]:
# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("="*70)
print("COLUMN TYPES")
print("="*70)
print(f"\nNumeric Columns ({len(numeric_cols)}): {numeric_cols}")
print(f"\nCategorical Columns ({len(categorical_cols)}): {categorical_cols}")

# Statistical summary for numeric columns
if len(numeric_cols) > 0:
    print("\n" + "="*70)
    print("STATISTICAL SUMMARY - NUMERIC COLUMNS")
    print("="*70)
    display(df[numeric_cols].describe().round(3))

# Frequency analysis for categorical columns
if len(categorical_cols) > 0:
    print("\n" + "="*70)
    print("CATEGORICAL COLUMNS - UNIQUE VALUES")
    print("="*70 + "\n")
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"{col:30s} - {unique_count:5d} unique values")
        if unique_count <= 10:
            print(f"  Values: {df[col].value_counts().to_dict()}")
        print()

## 4️⃣ Distribution Visualization for Numeric Features

In [ ]:
if len(numeric_cols) > 0:
    # Select key numeric columns for visualization (limit to 12)
    vis_cols = numeric_cols[:12]
    
    # Create distribution plots
    n_cols = min(4, len(vis_cols))
    n_rows = (len(vis_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 4))
    axes = axes.flatten() if len(vis_cols) > 1 else [axes]
    
    for idx, col in enumerate(vis_cols):
        if idx < len(axes):
            # Remove NaN values for plotting
            data = df[col].dropna()
            
            axes[idx].hist(data, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
            axes[idx].axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {data.mean():.2f}')
            axes[idx].axvline(data.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {data.median():.2f}')
            axes[idx].set_title(f'{col}\n(n={len(data):,})', fontweight='bold')
            axes[idx].set_xlabel('Value')
            axes[idx].set_ylabel('Frequency')
            axes[idx].legend()
            axes[idx].grid(True, alpha=0.3)
    
    # Hide unused subplots
    for idx in range(len(vis_cols), len(axes)):
        axes[idx].set_visible(False)
    
    plt.suptitle('Distribution of Numeric Features', fontsize=16, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.savefig('02_numeric_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 02_numeric_distributions.png")

## 5️⃣ Correlation Analysis

In [ ]:
if len(numeric_cols) >= 2:
    # Calculate correlation matrix
    correlation_matrix = df[numeric_cols].corr()
    
    # Create heatmap
    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(correlation_matrix), k=1)
    
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0,
                square=True,
                linewidths=1,
                cbar_kws={"shrink": 0.8},
                vmin=-1, vmax=1)
    
    plt.title('Correlation Matrix - Numeric Features', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('03_correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 03_correlation_matrix.png")
    
    # Find strong correlations
    print("\n" + "="*70)
    print("STRONG CORRELATIONS (|r| > 0.7)")
    print("="*70 + "\n")
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                strong_corr.append({
                    'Feature_1': correlation_matrix.columns[i],
                    'Feature_2': correlation_matrix.columns[j],
                    'Correlation': correlation_matrix.iloc[i, j]
                })
    
    if strong_corr:
        strong_corr_df = pd.DataFrame(strong_corr).sort_values('Correlation', key=abs, ascending=False)
        display(strong_corr_df)
    else:
        print("No strong correlations found.")

## 6️⃣ Outlier Detection & Analysis

In [ ]:
if len(numeric_cols) > 0:
    # Create boxplots for outlier detection
    vis_cols = numeric_cols[:12]
    n_cols = min(4, len(vis_cols))
    n_rows = (len(vis_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 4))
    axes = axes.flatten() if len(vis_cols) > 1 else [axes]
    
    outlier_summary = []
    
    for idx, col in enumerate(vis_cols):
        if idx < len(axes):
            data = df[col].dropna()
            
            # Calculate outliers using IQR method
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = data[(data < lower_bound) | (data > upper_bound)]
            outlier_pct = len(outliers) / len(data) * 100
            
            outlier_summary.append({
                'Feature': col,
                'Outlier_Count': len(outliers),
                'Outlier_Percentage': outlier_pct
            })
            
            # Create boxplot
            axes[idx].boxplot(data, vert=True)
            axes[idx].set_title(f'{col}\nOutliers: {len(outliers)} ({outlier_pct:.1f}%)', fontweight='bold')
            axes[idx].set_ylabel('Value')
            axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Hide unused subplots
    for idx in range(len(vis_cols), len(axes)):
        axes[idx].set_visible(False)
    
    plt.suptitle('Outlier Detection - Boxplots', fontsize=16, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.savefig('04_outlier_detection.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 04_outlier_detection.png")
    
    # Display outlier summary
    print("\n" + "="*70)
    print("OUTLIER SUMMARY")
    print("="*70 + "\n")
    outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier_Percentage', ascending=False)
    display(outlier_df)

## 7️⃣ Clustering Analysis (K-Means)

In [ ]:
if len(numeric_cols) >= 2:
    # Prepare data for clustering
    X = df[numeric_cols].fillna(df[numeric_cols].median())
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Elbow method to find optimal k
    inertias = []
    K_range = range(2, min(11, len(df)//10))
    
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
    
    # Plot elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
    plt.xlabel('Number of Clusters (k)', fontsize=12)
    plt.ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
    plt.title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('05_elbow_curve.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 05_elbow_curve.png")
    
    # Apply K-means with optimal k (default: 5)
    optimal_k = 5
    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    df['Cluster'] = kmeans.fit_predict(X_scaled)
    
    print(f"\n✓ Created {optimal_k} clusters")
    print("\nCluster Distribution:")
    display(df['Cluster'].value_counts().sort_index())

## 8️⃣ PCA Visualization

In [ ]:
if len(numeric_cols) >= 2 and 'Cluster' in df.columns:
    # Apply PCA for 2D visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Create PCA plot
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                         c=df['Cluster'], 
                         cmap='viridis', 
                         s=50, 
                         alpha=0.6,
                         edgecolors='black',
                         linewidth=0.5)
    
    plt.colorbar(scatter, label='Cluster')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
    plt.title('PCA Visualization - Cluster Analysis', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('06_pca_clusters.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 06_pca_clusters.png")
    
    print(f"\nPCA Explained Variance:")
    print(f"  PC1: {pca.explained_variance_ratio_[0]*100:.2f}%")
    print(f"  PC2: {pca.explained_variance_ratio_[1]*100:.2f}%")
    print(f"  Total: {sum(pca.explained_variance_ratio_)*100:.2f}%")

## 9️⃣ Cluster Profiling

In [ ]:
if 'Cluster' in df.columns and len(numeric_cols) > 0:
    # Calculate cluster profiles
    cluster_profiles = df.groupby('Cluster')[numeric_cols].mean().round(3)
    
    print("="*70)
    print("CLUSTER PROFILES - MEAN VALUES")
    print("="*70 + "\n")
    display(cluster_profiles.T)  # Transpose for better readability
    
    # Visualize cluster profiles
    top_features = numeric_cols[:8]  # Limit to 8 features for clarity
    cluster_profiles_subset = df.groupby('Cluster')[top_features].mean()
    
    # Normalize for heatmap
    cluster_profiles_norm = (cluster_profiles_subset - cluster_profiles_subset.min()) / (cluster_profiles_subset.max() - cluster_profiles_subset.min())
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(cluster_profiles_norm.T, 
                annot=cluster_profiles_subset.T.round(2), 
                fmt='.2f',
                cmap='YlOrRd', 
                cbar_kws={'label': 'Normalized Value'},
                linewidths=1,
                square=False)
    plt.xlabel('Cluster', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title('Cluster Profiles Heatmap (Top Features)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('07_cluster_profiles.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 07_cluster_profiles.png")

## 🔟 Categorical Analysis (if applicable)

In [ ]:
if len(categorical_cols) > 0:
    # Analyze top categorical columns
    vis_cats = [col for col in categorical_cols if df[col].nunique() <= 20][:6]
    
    if len(vis_cats) > 0:
        n_cols = min(3, len(vis_cats))
        n_rows = (len(vis_cats) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
        axes = axes.flatten() if len(vis_cats) > 1 else [axes]
        
        for idx, col in enumerate(vis_cats):
            if idx < len(axes):
                value_counts = df[col].value_counts().head(15)
                
                axes[idx].barh(range(len(value_counts)), value_counts.values, color='teal')
                axes[idx].set_yticks(range(len(value_counts)))
                axes[idx].set_yticklabels(value_counts.index)
                axes[idx].set_xlabel('Count', fontsize=11)
                axes[idx].set_title(f'{col}\n(Top 15 values)', fontweight='bold', fontsize=12)
                axes[idx].grid(True, alpha=0.3, axis='x')
                
                # Add value labels
                for i, v in enumerate(value_counts.values):
                    axes[idx].text(v + max(value_counts.values)*0.01, i, f'{v:,}', 
                                 va='center', fontsize=9)
        
        # Hide unused subplots
        for idx in range(len(vis_cats), len(axes)):
            axes[idx].set_visible(False)
        
        plt.suptitle('Categorical Features - Value Distribution', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig('08_categorical_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        print("\n✓ Saved: 08_categorical_analysis.png")

## 1️⃣1️⃣ Feature Importance (Random Forest)

In [ ]:
if 'Cluster' in df.columns and len(numeric_cols) >= 2:
    # Train Random Forest to find feature importance
    X_importance = df[numeric_cols].fillna(df[numeric_cols].median())
    y_importance = df['Cluster']
    
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_importance, y_importance)
    
    # Get feature importance
    importance_df = pd.DataFrame({
        'Feature': numeric_cols,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("="*70)
    print("FEATURE IMPORTANCE (Random Forest)")
    print("="*70 + "\n")
    display(importance_df.head(15))
    
    # Visualize top features
    plt.figure(figsize=(12, 8))
    top_n = min(15, len(importance_df))
    top_features = importance_df.head(top_n)
    
    plt.barh(range(len(top_features)), top_features['Importance'].values, color='darkgreen')
    plt.yticks(range(len(top_features)), top_features['Feature'].values)
    plt.xlabel('Importance Score', fontsize=12)
    plt.title(f'Top {top_n} Most Important Features', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig('09_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: 09_feature_importance.png")



## 1️⃣2️⃣ Export Final Results & Reports

In [ ]:
# Export enhanced dataset with clusters
output_file = 'gemini_analysis_complete.csv'
df.to_csv(output_file, index=False)
print(f"✓ Exported: {output_file}")
print(f"  - Total rows: {len(df):,}")
print(f"  - Total columns: {len(df.columns)}")

# Export cluster summary
if 'Cluster' in df.columns and len(numeric_cols) > 0:
    cluster_summary = df.groupby('Cluster').agg({
        **{col: ['mean', 'std', 'min', 'max'] for col in numeric_cols[:5]}
    }).round(3)
    cluster_summary.to_csv('gemini_cluster_summary.csv')
    print(f"\n✓ Exported: gemini_cluster_summary.csv")

# Export statistical summary
summary_stats = df[numeric_cols].describe().T
summary_stats.to_csv('gemini_statistical_summary.csv')
print(f"✓ Exported: gemini_statistical_summary.csv")

# Export correlation matrix
if len(numeric_cols) >= 2:
    correlation_matrix.to_csv('gemini_correlation_matrix.csv')
    print(f"✓ Exported: gemini_correlation_matrix.csv")

## 1️⃣3️⃣ Executive Summary Report

In [ ]:
from datetime import datetime

print("\n" + "#"*70)
print("# GEMINI - COMPREHENSIVE ANALYSIS REPORT")
print("#"*70 + "\n")

print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total Records: {len(df):,}")
print(f"Total Features: {len(df.columns)}")

print("\n" + "="*70)
print("DATA COMPOSITION")
print("="*70)
print(f"\nNumeric Features: {len(numeric_cols)}")
print(f"Categorical Features: {len(categorical_cols)}")

if len(numeric_cols) > 0:
    print("\n" + "="*70)
    print("KEY STATISTICS")
    print("="*70)
    for col in numeric_cols[:5]:
        print(f"\n{col}:")
        print(f"  Mean: {df[col].mean():.3f}")
        print(f"  Median: {df[col].median():.3f}")
        print(f"  Std Dev: {df[col].std():.3f}")
        print(f"  Range: [{df[col].min():.3f}, {df[col].max():.3f}]")

if 'Cluster' in df.columns:
    print("\n" + "="*70)
    print("CLUSTERING ANALYSIS")
    print("="*70)
    print(f"\nNumber of Clusters: {df['Cluster'].nunique()}")
    print("\nCluster Distribution:")
    for cluster, count in df['Cluster'].value_counts().sort_index().items():
        pct = count / len(df) * 100
        print(f"  Cluster {cluster}: {count:,} records ({pct:.1f}%)")

print("\n" + "="*70)
print("GENERATED OUTPUTS")
print("="*70)
print("\nCSV Files:")
print("  ✓ gemini_analysis_complete.csv")
print("  ✓ gemini_cluster_summary.csv")
print("  ✓ gemini_statistical_summary.csv")
print("  ✓ gemini_correlation_matrix.csv")

print("\nVisualizations:")
print("  ✓ 01_missing_data_analysis.png")
print("  ✓ 02_numeric_distributions.png")
print("  ✓ 03_correlation_matrix.png")
print("  ✓ 04_outlier_detection.png")
print("  ✓ 05_elbow_curve.png")
print("  ✓ 06_pca_clusters.png")
print("  ✓ 07_cluster_profiles.png")
print("  ✓ 08_categorical_analysis.png")
print("  ✓ 09_feature_importance.png")

print("\n" + "#"*70)
print("# ANALYSIS COMPLETE - ALL REPORTS GENERATED")
print("#"*70)